# *FEniCS tutorial:* Heat equation with Dirichlet boundary conditions in two dimensions

In this demo, we solve the two-dimensional diffusion equation with Dirichlet
boundary conditions $u_D$ and source term $f$. Both are chosen so as to yield
an exact analytic result against which we can compare the numerical results.

$$
\begin{align}
  u'   &= \nabla^2 u + f     \quad\text{in the unit square} \\
  u    &= u_D  \hphantom{u+f}\quad\text{on the boundary} \\
  u    &= u_0  \hphantom{u+f}\quad\;\text{at $t = 0$}
\end{align}
$$
with
$$
\begin{align}
  u_D  &= 1 + x^2 + \alpha y^2 + \beta t \\
  u_0 &= u_D(t=0) \\[0.5ex]
  f    &= \beta - 2 (1 + \alpha)
\end{align}
$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time as tm

# import os
# import sys
# import re
# from IPython.display import Image
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from fenics import *

In [ ]:
## set up the problem and define the main simulation function, evolve()

# simulation parameters
# -- boundary function
alpha = 3         # parameter alpha
beta = 1.2        # parameter beta
# -- time step
T = 2.0           # total simulation time
n_steps = 10      # number of time steps
dt = T / n_steps  # size of time step
nip = 1           # number of intervals between plots
# -- mesh density
nx = ny = 8       # mesh density along axes

# create mesh and define function space
mesh = UnitSquareMesh(nx, ny)
V = FunctionSpace(mesh, 'P', 1)

# define boundary condition
u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t',
                 degree=2, alpha=alpha, beta=beta, t=0)
u_D_min = 1.
u_D_max = 1. + 1 + alpha + beta * T

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u_D, boundary)

# define initial value
u_n = interpolate(u_D, V)

# define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(beta - 2 - 2*alpha)
F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)

# define time-evolution function
def evolve():

    # compute and report initial err at vertices
    t = 0
    u_D.t = t
    u_x = interpolate(u_D, V)
    err = np.abs(u_n.vector() - u_x.vector()).max()
    yield 0, u_n, err
    #print('idx = %2d: t = %.2f: err = %.3g' % (0, 0., err))

    # time-stepping
    u = Function(V)
    for n in range(1, n_steps + 1):

        # update current time
        t += dt
        u_D.t = t

        # compute solution
        solve(a == L, u, bc)

        # compute and report current state and max err at vertices
        if n % nip == 0:
            u_x = interpolate(u_D, V)
            err = np.abs(u.vector() - u_x.vector()).max()
            yield t, u, err
            #print('idx = %2d: t = %.2f: err = %.3g' % (n, t, err))

        # update previous solution
        u_n.assign(u)

In [ ]:
n_rows = 3
n_cols = 5
fig_wd = 15
# default sizing here yields unit aspect ratio
plt.figure(figsize = (fig_wd, fig_wd * n_rows // n_cols))

idx = 0
for t, u, e in evolve():
    idx += 1
    print('idx = %2d: t = %.2f: err = %.3g' % (idx, t, e))
    ax = plt.subplot(n_rows, n_cols, idx)
    plot(u, vmin=u_D_min, vmax=u_D_max)